<a href="https://colab.research.google.com/github/ownit4137/TIL/blob/main/DL%20from%20Scratch/1/210122_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 합성곱(Convolutional) 신경망

- 이미지 인식과 음성 인식 등에서 활용
- Convolutional - ReLU - Pooling으로 구성된 층으로 이루어짐(일반적인 층은 Affine - ReLU)

## Convolutional Layer

- 완전연결(Affine)계층은 데이터의 형상을 무시한 채 학습함
- 이미지 등의 데이터를 다룰 경우 형상이나 패턴을 인식하는 과정이 필요
- 입력 데이터에 필터(커널)를 곱하는 연산을 FMA라고 함(fused multiply-add)

### 패딩

- 합성곱 연산을 수행하기 전에 데이터 주변을 0으로 채우는 것
- *Convolution 연산 후 출력 데이터의 크기를 유지하기 위해*

### 스트라이드

- 필터를 적용하는 위치의 간격을 의미
- 출력 가로, 입력 가로, 필터 가로, 패딩, 스트라이드가 있을 때
- OW = (IW + 2P - FW) / S + 1, 정수로 반올림하여 사용

### 고차원 데이터

- 이미지와 같은 데이터는 RGB 채널 등 행렬의 차원이 커짐
- 이미지의 차원과 같은 차원의 필터를 사용
- 입력(C, H, W) * 필터(C, FH, FW) -> 출력(1, OH, OW)
- 입력(C, H, W) * 다중 필터(FN, C, FH, FW) -> 출력(FN, OH, OW)

### 배치 처리

- 여러 가지의 데이터를 한번에 처리(속도 면에서 이점을 가짐)
- 입력(N, C, H, W) * 다중 필터(FN, C, FH, FW) -> 출력(N, FN, OH, OW)

## Pooling Layer

- 지정한 영역을 원소 하나로 집약하여 공간의 크기를 줄임
- 최대 풀링 or 평균 풀링을 사용
- **입력 데이터에 사소한 차이가 있을 때 그 차이를 풀링으로 흡수할 수 있음**





# 코드

### im2col

- 합성곱 연산을 수행하기 위해서는 여러 개의 중첩 for문이 필요함
- 계산의 편의를 위해 im2col함수를 만들어 사용
- 4차원 데이터를 2차원으로 변환
- 필터를 곱하기 쉽게 입력 데이터가 reshape(-1, 필터 크기 * 채널) 형식으로 변형됨

In [4]:
from google.colab import drive 
drive.mount('/content/gdrive/')

%cd /content/gdrive/MyDrive/'Colab Notebooks'/

Mounted at /content/gdrive/
/content/gdrive/MyDrive/Colab Notebooks


In [6]:
def im2col(input_data, filter_h, filter_w, stride=1, pad=0):
    N, C, H, W = input_data.shape
    out_h = (H + 2*pad - filter_h)//stride + 1
    out_w = (W + 2*pad - filter_w)//stride + 1

    img = np.pad(input_data, [(0,0), (0,0), (pad, pad), (pad, pad)], 'constant')
    col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

    for y in range(filter_h):
        y_max = y + stride*out_h
        for x in range(filter_w):
            x_max = x + stride*out_w
            col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

    # 축이 주어졌을 때 축의 순서를 변경
    col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N*out_h*out_w, -1)
    return col

In [9]:
import numpy as np

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


### Convolution

In [ ]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = 1 + int((H + 2*self.pad - FH) / self.stride)
        out_w = 1 + int((W + 2*self.pad - FW) / self.stride)

        # 입력 데이터 변환 후 가중치 펼치기 -> 계산의 편의
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T 

        out = np.dot(col, col_W) + self.b
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)

        return out

### Pooling

In [ ]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
        self.x = None
        self.arg_max = None

    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1 + (H - self.pool_h) / self.stride)
        out_w = int(1 + (W - self.pool_w) / self.stride)

        # argmax를 이용하여 뽑아내기 좋게 변형
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)

        # axis=1 방향으로의 최댓값
        out = np.max(col, axis=1)
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)

        return out

# CNN 시각화

- 합성곱 신경망도 일반 신경망과 같이 데이터를 학습하여 필터의 가중치를 업데이트함
- 학습 후의 필터는 어떤 형상을 띄어 그 형상에 반응하게 됨
- 층이 깊어질 수록 형상의 수준이 높아짐(선 - 무늬 - 사물)

# 종류

- LeNet과 AlexNet이 있음

### LeNet

- 20년 전의 원조 CNN
- 활성화 함수로 sigmoid를 사용

### AlexNet

- 8년 전의 CNN
- 활성화 함수로 ReLU를 사용
- 국소적 정규화와 드롭아웃을 사용함